In [47]:
import pandas as pd
import psycopg2
import numpy as np
from sqlalchemy import create_engine

In [63]:
csv_file = "Medicare_Opioid_prescribing_rates.csv"
medicare_df = pd.read_csv(csv_file, thousands=",")
medicare_df = medicare_df[medicare_df.Prscrbr_Geo_Lvl == "State"]
medicare_df = medicare_df[medicare_df.Breakout == "Overall"]
medicare_df = medicare_df[medicare_df.Year == 2020]
medicare_df.head()

C:\Users\tammy\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (12,13,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Year,Prscrbr_Geo_Lvl,Prscrbr_Geo_Cd,Prscrbr_Geo_Desc,RUCA_Cd,Breakout_Type,Breakout,Tot_Prscrbrs,Tot_Opioid_Prscrbrs,Tot_Opioid_Clms,Tot_Clms,Opioid_Prscrbng_Rate,Opioid_Prscrbng_Rate_5Y_Chg,Opioid_Prscrbng_Rate_1Y_Chg,LA_Tot_Opioid_Clms,LA_Opioid_Prscrbng_Rate,LA_Opioid_Prscrbng_Rate_5Y_Chg,LA_Opioid_Prscrbng_Rate_1Y_Chg
3,2020,State,1.0,Alabama,NaN,Totals,Overall,15802.0,9576.0,1745736.0,28178642.0,6.20%,-1%,0%,126040.0,7.22%,-3%,0%
4,2020,State,2.0,Alaska,NaN,Totals,Overall,2824.0,1810.0,80501.0,1693029.0,4.75%,-2%,0%,13323.0,16.55%,-3%,0.36%
5,2020,State,4.0,Arizona,NaN,Totals,Overall,26427.0,16128.0,1299090.0,26247794.0,4.95%,-2%,0%,166918.0,12.85%,-4%,-1%
6,2020,State,5.0,Arkansas,NaN,Totals,Overall,9902.0,7099.0,954089.0,17659571.0,5.40%,-1%,0%,68301.0,7.16%,-3%,0%
7,2020,State,6.0,California,NaN,Totals,Overall,130251.0,78859.0,5524000.0,137442284.0,4.02%,-1%,0%,567386.0,10.27%,-2%,0%


In [64]:
csv_file = "Medicaid_Opioid_prescribing_rates.csv"
medicaid_df = pd.read_csv(csv_file, thousands=",")
medicaid_df = medicaid_df[medicaid_df.Geo_Lvl == "State"]
medicaid_df = medicaid_df[medicaid_df.Plan_Type == "All"]
medicaid_df = medicaid_df[medicaid_df.Year == 2020]
medicaid_df = medicaid_df.dropna()
medicaid_df.head()

,Year,Geo_Lvl,Geo_Cd,Geo_Desc,Plan_Type,Tot_Opioid_Clms,Tot_Clms,Opioid_Prscrbng_Rate,Opioid_Prscrbng_Rate_5Y_Chg,Opioid_Prscrbng_Rate_1Y_Chg,LA_Tot_Opioid_Clms,LA_Opioid_Prscrbng_Rate,LA_Opioid_Prscrbng_Rate_5Y_Chg,LA_Opioid_Prscrbng_Rate_1Y_Chg
3,2020,State,1.0,Alabama,All,182840.0,6750173.0,2.71,-2.92,-0.39,8438.0,4.62,-0.16,0.02
6,2020,State,2.0,Alaska,All,57160.0,1332333.0,4.29,-3.32,-0.27,7269.0,12.72,0.40,0.08
9,2020,State,4.0,Arizona,All,496944.0,13520919.0,3.68,-4.11,-0.31,35666.0,7.18,-3.21,-0.26
12,2020,State,5.0,Arkansas,All,183044.0,4723341.0,3.88,-3.13,-0.44,6304.0,3.44,-1.35,-0.32
15,2020,State,6.0,California,All,2254586.0,75111185.0,3.00,-2.59,-0.20,140342.0,6.22,-0.29,-0.08


In [65]:
csv_file = "SUDORS_Fatal_Overdose_Data.csv"
Fatal_OD_df = pd.read_csv(csv_file)
Fatal_OD_df = Fatal_OD_df[Fatal_OD_df.year == 2020]
Fatal_OD_df.head()

,Jurisdiction,year,alldrug_deaths,alldrug_rate,opioids_deaths,opioids_rate,imfs_deaths,imfs_rate,heroin_deaths,heroin_rate,...,recentrelease_percent,curr_paintrt_deaths,curr_paintrt_percent,homeless_deaths,homeless_percent,naloxone_deaths,naloxone_percent,returnopioids_deaths,returnopioids_percent,Unnamed: 172
0,Alaska,2020,172,23.9,112,15.5,58,8.0,48,6.7,...,7.0,34,19.8,12,7.1,20,11.6,2,1.2,NaN
1,Arizona,2020,2405,33.9,1857,26.1,1463,20.6,284,4.0,...,11.5,219,9.2,383,16.1,641,26.9,126,5.3,NaN
2,Colorado,2020,1327,22.4,974,16.3,557,9.3,276,4.6,...,9.5,145,11.5,134,10.7,187,15.7,123,9.8,NaN
3,Connecticut,2020,1374,39.4,1278,36.7,1158,33.4,261,7.3,...,9.8,117,8.5,62,4.5,425,31.0,75,5.5,NaN
4,Delaware,2020,445,47.3,416,44.4,371,39.9,105,11.3,...,21.0,50,11.3,53,12.0,52,11.8,61,13.8,NaN


In [66]:
Fatal_OD_df.count()

Jurisdiction             33
year                     33
alldrug_deaths           33
alldrug_rate             33
opioids_deaths           33
                         ..
naloxone_deaths          33
naloxone_percent         33
returnopioids_deaths     33
returnopioids_percent    33
Unnamed: 172              0
Length: 173, dtype: int64

In [67]:
# Create a filtered dataframe from specific columns
Fatal_OD_cols = ['Jurisdiction', 'year', 'alldrug_deaths','opioids_deaths','opioids_rate','rxopioids_deaths','rxopioids_rate','opioids_percent','male_deaths', 'male_percent','female_deaths','female_percent', 'black_nh_deaths','black_nh_percent','white_nh_deaths',
'white_nh_percent','hisp_deaths','hisp_percent']
Fatal_OD_df_transformed= Fatal_OD_df[Fatal_OD_cols].copy()

# Rename the column headers
Fatal_OD_df_transformed = Fatal_OD_df_transformed.rename(columns={'Jurisdiction': 'id'})

# Clean the data by dropping duplicates and setting the index
Fatal_OD_df_transformed.drop_duplicates('id', inplace=True)
Fatal_OD_df_transformed.set_index('id', inplace=True)

Fatal_OD_df_transformed.head()

,year,alldrug_deaths,opioids_deaths,opioids_rate,rxopioids_deaths,rxopioids_rate,opioids_percent,male_deaths,male_percent,female_deaths,female_percent,black_nh_deaths,black_nh_percent,white_nh_deaths,white_nh_percent,hisp_deaths,hisp_percent
id,,,,,,,,,,,,,,,,,
Alaska,2020,172,112,15.5,47,6.6,65.1,111,64.5,61,35.5,11,6.6,87,52.1,3,1.8
Arizona,2020,2405,1857,26.1,303,4.2,77.2,1766,73.4,639,26.6,197,8.3,1335,56.0,701,29.4
Colorado,2020,1327,974,16.3,399,6.7,73.4,944,71.1,383,28.9,98,7.5,834,63.4,337,25.6
Connecticut,2020,1374,1278,36.7,326,9.1,93.0,1032,75.1,342,24.9,190,13.9,934,68.4,228,16.7
Delaware,2020,445,416,44.4,78,8.1,93.5,305,68.5,140,31.5,87,19.6,332,74.8,22,5.0


In [68]:
Fatal_OD_df_transformed.count()

year                33
alldrug_deaths      33
opioids_deaths      33
opioids_rate        33
rxopioids_deaths    33
rxopioids_rate      33
opioids_percent     33
male_deaths         33
male_percent        33
female_deaths       33
female_percent      33
black_nh_deaths     33
black_nh_percent    33
white_nh_deaths     33
white_nh_percent    33
hisp_deaths         33
hisp_percent        33
dtype: int64

In [69]:
 # Create a filtered dataframe from specific columns
medicaid_cols = ['Geo_Desc','Year','Plan_Type','Tot_Opioid_Clms','LA_Tot_Opioid_Clms','LA_Opioid_Prscrbng_Rate',
'LA_Opioid_Prscrbng_Rate_5Y_Chg','LA_Opioid_Prscrbng_Rate_1Y_Chg']
medicaid_df_transformed= medicaid_df[medicaid_cols].copy()
medicaid_df_transformed.count()
medicaid_df_transformed.head()

# Rename the column headers
medicaid_df_transformed = medicaid_df_transformed.rename(columns={'Geo_Desc': 'id'})
medicaid_df_transformed = medicaid_df_transformed.rename(columns={'Year': 'year'})
medicaid_df_transformed = medicaid_df_transformed.rename(columns={'Year': 'year', 
'Plan_Type':'plan_type','Tot_Opioid_Clms':'tot_opioid_clms', 'LA_Tot_Opioid_Clms':'la_tot_opioid_clms',
'LA_Opioid_Prscrbng_Rate':'la_opioid_prscrbng_rate',
'LA_Opioid_Prscrbng_Rate_5Y_Chg':'la_opioid_prscrbng_rate_5y_chg',
'LA_Opioid_Prscrbng_Rate_1Y_Chg':'la_opioid_prscrbng_rate_1y_chg'})

# Clean the data by dropping duplicates and setting the index
medicaid_df_transformed.drop_duplicates('id', inplace=True)
medicaid_df_transformed.set_index('id', inplace=True)
medicaid_df_transformed.head()


,year,plan_type,tot_opioid_clms,la_tot_opioid_clms,la_opioid_prscrbng_rate,la_opioid_prscrbng_rate_5y_chg,la_opioid_prscrbng_rate_1y_chg
id,,,,,,,
Alabama,2020,All,182840.0,8438.0,4.62,-0.16,0.02
Alaska,2020,All,57160.0,7269.0,12.72,0.40,0.08
Arizona,2020,All,496944.0,35666.0,7.18,-3.21,-0.26
Arkansas,2020,All,183044.0,6304.0,3.44,-1.35,-0.32
California,2020,All,2254586.0,140342.0,6.22,-0.29,-0.08


In [70]:
medicaid_df_transformed.count()

year                              51
plan_type                         51
tot_opioid_clms                   51
la_tot_opioid_clms                51
la_opioid_prscrbng_rate           51
la_opioid_prscrbng_rate_5y_chg    51
la_opioid_prscrbng_rate_1y_chg    51
dtype: int64

In [71]:
 # Create a filtered dataframe from specific columns
medicare_cols = ['Prscrbr_Geo_Desc', 'Year','Breakout','Tot_Opioid_Clms','LA_Tot_Opioid_Clms','LA_Opioid_Prscrbng_Rate',
'LA_Opioid_Prscrbng_Rate_5Y_Chg','LA_Opioid_Prscrbng_Rate_1Y_Chg']
medicare_df_transformed= medicare_df[medicare_cols].copy()
# Rename the column headers
medicare_df_transformed = medicare_df_transformed.rename(columns={'Prscrbr_Geo_Desc': 'id'})
medicare_df_transformed = medicare_df_transformed.rename(columns={'Year': 'year', 
'Breakout':'breakout','Tot_Opioid_Clms':'tot_opioid_clms', 'LA_Tot_Opioid_Clms':'la_tot_opioid_clms',
'LA_Opioid_Prscrbng_Rate':'la_opioid_prscrbng_rate',
'LA_Opioid_Prscrbng_Rate_5Y_Chg':'la_opioid_prscrbng_rate_5y_chg',
'LA_Opioid_Prscrbng_Rate_1Y_Chg':'la_opioid_prscrbng_rate_1y_chg'})

# Clean the data by dropping duplicates, removing percent sign and setting the index
medicare_df_transformed.drop_duplicates('id', inplace=True)
medicare_df_transformed = medicare_df_transformed.replace({'%': ''}, regex=True)
medicare_df_transformed.set_index('id', inplace=True)

medicare_df_transformed.head()

,year,breakout,tot_opioid_clms,la_tot_opioid_clms,la_opioid_prscrbng_rate,la_opioid_prscrbng_rate_5y_chg,la_opioid_prscrbng_rate_1y_chg
id,,,,,,,
Alabama,2020,Overall,1745736.0,126040.0,7.22,-3,0
Alaska,2020,Overall,80501.0,13323.0,16.55,-3,0.36
Arizona,2020,Overall,1299090.0,166918.0,12.85,-4,-1
Arkansas,2020,Overall,954089.0,68301.0,7.16,-3,0
California,2020,Overall,5524000.0,567386.0,10.27,-2,0


In [72]:
medicare_df_transformed.count()

year                              55
breakout                          55
tot_opioid_clms                   55
la_tot_opioid_clms                54
la_opioid_prscrbng_rate           54
la_opioid_prscrbng_rate_5y_chg    54
la_opioid_prscrbng_rate_1y_chg    54
dtype: int64

In [73]:
#Connect to PGAdmin
protocol = 'postgresql'
username = 'tammy'
password = 'Mar152012!'
host = 'localhost'
port = 5432
database_name = 'Project 2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}/{database_name}'
engine = create_engine(rds_connection_string)

In [74]:
# Confirm tables
engine.table_names()

['medicare_opioid_prescribing_rate',
 'medicaid_opioid_prescribing_rate',
 'sudors_fatal_overdose_data']

In [75]:
#Load DataFrames into database
medicare_df_transformed.to_sql(name='medicare_opioid_prescribing_rate', con=engine, if_exists='append', index=True)

In [76]:
#Load DataFrames into database
medicaid_df_transformed.to_sql(name='medicaid_opioid_prescribing_rate', con=engine, if_exists='append', index=True)

In [77]:
#Load DataFrames into database
Fatal_OD_df_transformed.to_sql(name='sudors_fatal_overdose_data', con=engine, if_exists='append', index=True)